<a href="https://colab.research.google.com/github/adrian-lison/gnn-community-detection/blob/master/Notebooks/LGNN_Semi_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model #4 LGNN
----------


In [13]:
#!pip install dgl

## Imports

In [14]:
# dgl
import dgl
import dgl.function as fn
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

# pytorch
import torch as th
import torch.nn as nn
import torch.nn.functional as F

# other
import time
import numpy as np
import random as rng
import scipy.sparse as ss
import networkx as nx

## GNN Definition

In [15]:
# Return a list containing features gathered from multiple radius.
import dgl.function as fn
def aggregate_radius(radius, g, z):
    # initializing list to collect message passing result
    z_list = []
    g.ndata['z'] = z
    # pulling message from 1-hop neighbourhood
    g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
    z_list.append(g.ndata['z'])
    for i in range(radius - 1):
        for j in range(2 ** i):
            #pulling message from 2^j neighborhood
            g.update_all(fn.copy_src(src='z', out='m'), fn.sum(msg='m', out='z'))
        z_list.append(g.ndata['z'])
    return z_list

In [16]:
class LGNNCore(nn.Module):
    def __init__(self, in_feats, out_feats, radius):
        super(LGNNCore, self).__init__()
        self.out_feats = out_feats
        self.radius = radius

        self.linear_prev = nn.Linear(in_feats, out_feats)
        self.linear_deg = nn.Linear(in_feats, out_feats)
        self.linear_radius = nn.ModuleList(
                [nn.Linear(in_feats, out_feats) for i in range(radius)])
        self.linear_fuse = nn.Linear(in_feats, out_feats)
        self.bn = nn.BatchNorm1d(out_feats)

    def forward(self, g, feat_a, feat_b, deg, pm_pd):
        # term "prev"
        prev_proj = self.linear_prev(feat_a)
        # term "deg"
        deg_proj = self.linear_deg(deg * feat_a)

        # term "radius"
        # aggregate 2^j-hop features
        hop2j_list = aggregate_radius(self.radius, g, feat_a)
        # apply linear transformation
        hop2j_list = [linear(x) for linear, x in zip(self.linear_radius, hop2j_list)]
        radius_proj = sum(hop2j_list)

        # term "fuse"
        fuse = self.linear_fuse(th.mm(pm_pd, feat_b))

        # sum them together
        result = prev_proj + deg_proj + radius_proj + fuse

        # skip connection and batch norm
        n = self.out_feats // 2
        result = th.cat([result[:, :n], F.relu(result[:, n:])], 1)
        result = self.bn(result)

        return result

In [17]:
class LGNNLayer(nn.Module):
    def __init__(self, in_feats, out_feats, radius):
        super(LGNNLayer, self).__init__()
        self.g_layer = LGNNCore(in_feats, out_feats, radius)
        self.lg_layer = LGNNCore(in_feats, out_feats, radius)

    def forward(self, g, lg, x, lg_x, deg_g, deg_lg, pm_pd):
        next_x = self.g_layer(g, x, lg_x, deg_g, pm_pd)
        pm_pd_y = th.transpose(pm_pd, 0, 1)
        next_lg_x = self.lg_layer(lg, lg_x, x, deg_lg, pm_pd_y)
        return next_x, next_lg_x

In [18]:
class LGNN(nn.Module):
    def __init__(self, radius):
        super(LGNN, self).__init__()
        self.layer1 = LGNNLayer(1, 16, radius)  # input is scalar feature
        self.layer2 = LGNNLayer(16, 16, radius)  # hidden size is 16
        self.layer3 = LGNNLayer(16, 16, radius)
        self.linear = nn.Linear(16, 7)  # predice seven classes

    def forward(self, g, lg, pm_pd):
        # compute the degrees
        deg_g = g.in_degrees().float().unsqueeze(1)
        deg_lg = lg.in_degrees().float().unsqueeze(1)
        # use degree as the input feature
        x, lg_x = deg_g, deg_lg
        x, lg_x = self.layer1(g, lg, x, lg_x, deg_g, deg_lg, pm_pd)
        x, lg_x = self.layer2(g, lg, x, lg_x, deg_g, deg_lg, pm_pd)
        x, lg_x = self.layer3(g, lg, x, lg_x, deg_g, deg_lg, pm_pd)
        return self.linear(x)

## Data Loading

In [19]:
#Loading CORA
data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labelsCORA = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph
g2 = data.graph
# removing doesnt work
#g.remove_edges_from(g.selfloop_edges())
g = DGLGraph(g)
#g.add_edges(g.nodes(), g.nodes()) #What does this do?
print('We have %d nodes. We have %d edges. Before DGL Graph adding edges: %d' % (g.number_of_nodes(),g.number_of_edges(),g2.number_of_edges()))

We have 2708 nodes. We have 10556 edges. Before DGL Graph adding edges: 10556


In [20]:
edges_per_node = {}
for x in g2.adj.items():
  z = [] 
  for i in x[1]:
    z.append(i)
  edges_per_node[x[0]] = z
edges_per_node[0]
len(edges_per_node)

2708

In [22]:
edges_per_node[2]

[410, 471, 552, 565]

## Select Training Set

In [0]:
from scipy.sparse import lil_matrix
matrix_p2 = lil_matrix((g2.number_of_nodes(),g2.number_of_edges()))
for i in range(len(edges_per_node)):
  matrix_p2[i,edges_per_node[i]] = 1
new_pdw = ss.coo_matrix(matrix_p2,dtype="int64")
new_pdw

<2708x10556 sparse matrix of type '<class 'numpy.int64'>'
	with 10556 stored elements in COOrdinate format>

In [0]:
def get_label(x):
  return(int(labelsCORA[x]))

#Percentage of nodes known for training
percentage_open_nodes = 0.1
graph_node_size = int(g.number_of_nodes())
open_nodes = rng.sample(range(0,graph_node_size),int(graph_node_size*percentage_open_nodes))
open_nodes_labels = list(map(get_label,open_nodes))
len(open_nodes)

270

In [0]:
inputs = torch.eye(graph_node_size)
labeled_nodes = torch.tensor(open_nodes)  #
labels = torch.tensor(open_nodes_labels)  # 

In [0]:
#inputs_pmpd = nx.to_scipy_sparse_matrix(g2,dtype="int64",  format='coo')
inputs_pmpd = new_pdw

In [ ]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [0]:
#Helper to print accuracy
def output_accuracy2(i,logits0):
  all_logits = logits0
  labels_final = []

  for v in range(graph_node_size):
    calculated_label = all_logits[i][v].numpy().argmax()
    labels_final.append(calculated_label)
  n_correct = 0
  n_correct_random = 0
  for v in range(graph_node_size):
    if (labels_final[v] == int(labelsCORA[v])):
      n_correct +=1
    if (int(rng.randrange(0,6)) == int(labelsCORA[v])):
      n_correct_random +=1  
  return(percentage_open_nodes,(n_correct/graph_node_size),(n_correct-len(open_nodes))/(graph_node_size-len(open_nodes)))

In [0]:
# create the model
net = LGNN(radius=3)
# define the optimizer
optimizer = th.optim.Adam(model.parameters(), lr=1e-2)

# a util function to convert a scipy.coo_matrix to torch.SparseFloat
def sparse2th(mat):
    value = mat.data
    indices = th.LongTensor([mat.row, mat.col])
    tensor = th.sparse.FloatTensor(indices, th.from_numpy(value).float(), mat.shape)
    return tensor

all_logits = []

# train
for epoch in range(100):

    pmpd = sparse2th(inputs_pmpd)
    lg = g.line_graph(backtracking=False)

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = F.log_softmax(net(g, lg, pmpd),1)
    val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    net.train()

    logits = net(g, lg, pmpd)
    # Save logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)

    # Compute loss for train nodes
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch %d | Loss: %.4f | Total: %.4f' % (epoch, loss.item(), val_loss.item()))

Epoch 0 | Loss: 1.9892 | Total: 2.0069 | Accuracy: 0.1540 | Accuracy excluding open nodes: 0.0603 | Open nodes: 0
Epoch 1 | Loss: 1.9219 | Total: 1.9453 | Accuracy: 0.1592 | Accuracy excluding open nodes: 0.0660 | Open nodes: 0
Epoch 2 | Loss: 1.8484 | Total: 1.8737 | Accuracy: 0.2585 | Accuracy excluding open nodes: 0.1764 | Open nodes: 0
Epoch 3 | Loss: 1.7489 | Total: 1.7725 | Accuracy: 0.3283 | Accuracy excluding open nodes: 0.2539 | Open nodes: 0
Epoch 4 | Loss: 1.7265 | Total: 1.7530 | Accuracy: 0.3039 | Accuracy excluding open nodes: 0.2268 | Open nodes: 0
Epoch 5 | Loss: 1.6909 | Total: 1.7279 | Accuracy: 0.3353 | Accuracy excluding open nodes: 0.2617 | Open nodes: 0
Epoch 6 | Loss: 1.6665 | Total: 1.7025 | Accuracy: 0.3486 | Accuracy excluding open nodes: 0.2765 | Open nodes: 0
Epoch 7 | Loss: 1.6450 | Total: 1.6849 | Accuracy: 0.3744 | Accuracy excluding open nodes: 0.3052 | Open nodes: 0
Epoch 8 | Loss: 1.6231 | Total: 1.6656 | Accuracy: 0.4066 | Accuracy excluding open node

## Evaluation